<a href="https://colab.research.google.com/github/ChanaPark/osfinal/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C_%EB%B0%95%EC%B0%AC%EC%95%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! wget -q -O - ipv4.icanhazip.com

35.204.67.32


In [ ]:
%%writefile app.py

Overwriting app.py


In [ ]:
!ps aux | grep streamlit

root       77500  0.0  0.0   6484  2276 ?        S    12:28   0:00 grep streamlit


In [5]:
!streamlit cache clear
!streamlit run app.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.204.67.32:8501

your url is: https://early-sides-dress.loca.lt
  Stopping...
^C


<웹크롤링>

In [ ]:
from bs4 import BeautifulSoup

url = 'https://nct.go.kr/distMental/rating/rating02_2.do'
response = requests.get(url)

html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')

questions = [td.get_text(strip=True) for td in soup.find_all('td', class_='tLeft')]

print(questions)

['일 또는 여가 활동을 하는데 흥미나 즐거움을 느끼지 못함', '기분이 가라앉거나.우울하거나.희망이 없음', '잠이 들거나 계속 잠을 자는 것이 어려움. 또는 잠을 너무 많이 잠', '피곤하다고 느끼거나 기운이 거의 없음', '입맛이 없거나 과식을 함', '자신을 부정적으로 봄 – 혹은 자신이 실패자라고 느끼거나 자신 또는 가족을 실망시킴', '신문을 읽거나 텔레비전 보는 것과 같은 일에 집중하는 것이 어려움', '다른 사람들이 주목할 정도로 너무 느리게 움직이거나 말을 함또는 반대로 평상시보다 많이 움직여서, 너무 안절부절 못하거나 들떠 있음', '자신이 죽는 것이 더 낫다고 생각하거나 어떤 식으로든 자신을 해칠것이라고 생각함']


In [ ]:
import json

# 질문 데이터를 JSON 파일로 저장
data = {
    "questions": questions
}

with open("questions.json", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("JSON 파일 저장 완료!")

JSON 파일 저장 완료!


In [ ]:
'''
app.py 코드

import google.generativeai as genai
import streamlit as st
import requests
from bs4 import BeautifulSoup
import json

# API 설정
YOUR_API_KEY = '00000'
genai.configure(api_key=YOUR_API_KEY)

st.title("상담Chat: the Counselor for You")

# 모델 로드 함수
@st.cache_resource
def load_model():
    generation_config = genai.GenerationConfig(max_output_tokens=1000)
    model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
    st.markdown("model loaded...")
    return model

model = load_model()

def load_questions():
    with open("questions.json", "r", encoding="utf-8") as f:
        data = json.load(f)
        return data["questions"]

questions = load_questions()
total_score = 0

def conduct_diagnosis():
    global total_score
    st.markdown("지난 2주 간 다음의 증상에 얼마나 자주 해당되었는지 응답해주세요.")
    st.markdown("전혀 해당되지 않았다면 0, 가끔 해당되었다면 1, 7일 이상 해당되었다면 2, 매일 해당되었다면 3을 입력해주세요.")

    for i, question in enumerate(questions):
        while True:
            answer = input(f"{i+1}. {question}: ")
            if answer in ["0", "1", "2", "3"]:
                total_score += int(answer)
                break
            else:
              st.markdown("0, 1, 2, 3 중 하나를 입력해주세요.")

def provide_feedback(total_score):
    if total_score <= 4:
        st.markdown("총점:", total_score, "점. 정상적인 수준입니다.")
    elif total_score <= 9:
        st.markdown("총점:", total_score, "점. 경미한 수준의 우울 증상이 나타날 수 있습니다.")
    elif total_score <= 14:
        st.markdown("총점:", total_score, "점. 중간 수준의 우울 증상이 나타날 수 있습니다.")
    elif total_score <= 19:
        st.markdown("총점은", total_score, "점으로 약간 심한 수준의 우울 증상이 나타날 수 있습니다. 전문가와 상담을 권장합니다.")
    else:
        st.markdown("총점은", total_score, "점으로 아주 심한 수준의 우울 증상이 나타날 수 있습니다. 반드시 전문가와 상담하세요.")
    st.markdown("진단을 종료합니다.")

# 대화 세션 초기화 (시스템 메시지 제거)
if "chat_session" not in st.session_state:
    st.session_state["chat_session"] = model.start_chat(history=[{
      "role": "user", "parts": ["당신은 심리적으로 공감하고 현실적인 대책을 궁리해주는 상담사입니다. 사용자에게 도움을 주세요."]
    }])
    st.session_state.messages = []
    st.session_state.language = "한국어"
    st.session_state.stage = 1

# 시스템 메시지 화면에 출력
st.markdown("**안녕하세요! 심리적으로 어려운 점이 있으시면 언제든지 말씀해주세요. 함께 해결해 나갈 수 있도록 노력하겠습니다.**")

# 대화 기록 표시
for content in st.session_state.chat_session.history:
    with st.chat_message("ai" if content.role == "model" else "user"):
        st.markdown(content.parts[0].text)

if "diagnosis_started" not in st.session_state:
    st.session_state.diagnosis_started = False

# 사용자 입력 처리
if prompt := st.chat_input("무엇이 고민인가요?"):
    with st.chat_message("user"):
        st.markdown(prompt)
    with st.chat_message("ai", avatar="😺"):
        response = st.session_state.chat_session.send_message(prompt)
        st.markdown(response.text)
    if "우울" in prompt:
        st.markdown("우울증상이 계속된다면 테스트로 자가진단을 실행해보세요.")
        if st.button("자가진단 진행"):
            st.session_state.diagnosis_started = True  # 상태 업데이트

if st.session_state.diagnosis_started:
    try:
        st.markdown("자가진단을 시작합니다.")
        conduct_diagnosis()
        feedback = provide_feedback(total_score)
        st.write(feedback)
        st.session_state.diagnosis_started = False  # 완료 후 상태 초기화
    except Exception as e:
        st.error(f"오류가 발생했습니다: {e}")
'''

In [ ]:
'''
import google.generativeai as genai
import streamlit as st
import requests
from bs4 import BeautifulSoup

# API 설정
YOUR_API_KEY = 'key'
genai.configure(api_key=YOUR_API_KEY)

st.title("상담Chat: the Counselor for You")

@st.cache_resource
def load_model():
    """
    제미나이 모델을 로드합니다.
    """
    model = genai.GenerativeModel(
      model_name='gemini-1.5-chat',
      temperature=0.5,
      max_output_tokens=500)
    print("모델이 로드되었습니다.")
    return model

model = load_model()

# 대화 세션 초기화
if "chat_session" not in st.session_state:
    st.session_state["chat_session"] = genai.start_chat(
        model="gemini-1.5-chat",  # 모델 이름 지정
        temperature=0.5,         # 응답의 창의성 제어
        max_output_tokens=500,   # 응답의 최대 토큰 수
        history=[
            {"role": "system", "content": "당신은 심리적으로 공감하고 현실적인 대책을 궁리해주는 상담사입니다. 사용자에게 도움을 주세요."}
        ]
    )
# 대화 기록 표시
for content in st.session_state.chat_session.history:
    with st.chat_message("상담사" if content['role'] == "model" else "user"):
        st.markdown(content['content'])

# 사용자 입력 처리
if prompt := st.chat_input("메시지를 입력하세요."):
    with st.chat_message("user"):
        st.markdown(prompt)
    with st.chat_message("상담사"):
        response = st.session_state.chat_session.send_message(
        message=prompt,
        temperature=0.5,
        max_output_tokens=500
        )
        st.markdown(response['content'])

# 웹크롤링 함수
@st.cache_resource
def fetch_counseling_articles():
    """
    국립 정신건강 센터에서 심리상담 관련 정보 크롤링
    """
    url = ["https://nct.go.kr/distMental/rating/rating02_2.do",
            "https://www.mohw.go.kr/menu.es?mid=a10706040100",
            ""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    print("심리적인 어려움을 혼자 해결하기 어렵다면 전문가와의 상담을 추천합니다.")
    print("국립정신건강센터 등 전문 기관에서 상담을 받아보세요.")
    print("더 자세한 정보는 다음 링크에서 확인할 수 있습니다:", url)

if __name__ == "__main__":
    recommend_professional_counseling()
    '''

'\nimport google.generativeai as genai\nimport streamlit as st\nimport requests\nfrom bs4 import BeautifulSoup\n\n# API 설정\nYOUR_API_KEY = \'key\'\ngenai.configure(api_key=YOUR_API_KEY)\n\nst.title("상담Chat: the Counselor for You")\n\n@st.cache_resource\ndef load_model():\n    """\n    제미나이 모델을 로드합니다.\n    """\n    model = genai.GenerativeModel(\n      model_name=\'gemini-1.5-chat\',\n      temperature=0.5,\n      max_output_tokens=500)\n    print("모델이 로드되었습니다.")\n    return model\n\nmodel = load_model()\n\n# 대화 세션 초기화\nif "chat_session" not in st.session_state:\n    st.session_state["chat_session"] = genai.start_chat(\n        model="gemini-1.5-chat",  # 모델 이름 지정\n        temperature=0.5,         # 응답의 창의성 제어\n        max_output_tokens=500,   # 응답의 최대 토큰 수\n        history=[\n            {"role": "system", "content": "당신은 심리적으로 공감하고 현실적인 대책을 궁리해주는 상담사입니다. 사용자에게 도움을 주세요."}\n        ]\n    )\n# 대화 기록 표시\nfor content in st.session_state.chat_session.history:\n    with st.chat_message("상담사

In [ ]:
'''
import google.generativeai as genai
import streamlit as st

# genai.configure(api_key=GOOGLE_API_KEY)
YOUR_API_KEY = 'key'
genai.configure(api_key=YOUR_API_KEY)

st.title("Gemini-Bot")

@st.cache_resource
def load_model():
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("model loaded...")
    return model

model = load_model()

if "chat_session" not in st.session_state:
    st.session_state["chat_session"] = model.start_chat(history=[])

for content in st.session_state.chat_session.history:
    with st.chat_message("ai" if content.role == "model" else "user"):
        st.markdown(content.parts[0].text)

if prompt := st.chat_input("메시지를 입력하세요."):
    with st.chat_message("user"):
        st.markdown(prompt)
    with st.chat_message("ai"):
        response = st.session_state.chat_session.send_message(prompt)
        st.markdown(response.text)
        '''

'\nimport google.generativeai as genai\nimport streamlit as st\n\n# genai.configure(api_key=GOOGLE_API_KEY)\nYOUR_API_KEY = \'key\'\ngenai.configure(api_key=YOUR_API_KEY)\n\nst.title("Gemini-Bot")\n\n@st.cache_resource\ndef load_model():\n    model = genai.GenerativeModel(\'gemini-1.5-flash\')\n    print("model loaded...")\n    return model\n\nmodel = load_model()\n\nif "chat_session" not in st.session_state:    \n    st.session_state["chat_session"] = model.start_chat(history=[])\n\nfor content in st.session_state.chat_session.history:\n    with st.chat_message("ai" if content.role == "model" else "user"):\n        st.markdown(content.parts[0].text)\n\nif prompt := st.chat_input("메시지를 입력하세요."):    \n    with st.chat_message("user"):\n        st.markdown(prompt)\n    with st.chat_message("ai"):\n        response = st.session_state.chat_session.send_message(prompt)        \n        st.markdown(response.text)\n        '

In [ ]:
'''import google.generativeai as genai
import streamlit as st
import requests
from bs4 import BeautifulSoup

# API 설정
YOUR_API_KEY = 'key'
genai.configure(api_key=YOUR_API_KEY)

st.title("상담Chat: the Counselor for You")

# 모델 로드 함수
@st.cache_resource
def load_model():
    generation_config = genai.GenerationConfig(max_output_tokens=500)
    model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
    print("model loaded...")
    return model

model = load_model()

# 대화 세션 초기화
if "chat_session" not in st.session_state:
    st.session_state["chat_session"] = model.start_chat(
        history=[
            {"ai": "system", "content": "당신은 심리적으로 공감하고 현실적인 대책을 궁리해주는 상담사입니다. 사용자에게 도움을 주세요."}
        ]
    )

# 대화 기록 표시
for content in st.session_state.chat_session.history:
    with st.chat_message("ai" if content.role == "model" else "user"):
        st.markdown(content.parts[0].text)

# 사용자 입력 처리
if prompt := st.chat_input("무엇이 고민인가요?"):
    with st.chat_message("user"):
        st.markdown(prompt)
    with st.chat_message("ai"):
        response = st.session_state.chat_session.send_message(prompt)
        st.markdown(response.text)'''

'import google.generativeai as genai\nimport streamlit as st\nimport requests\nfrom bs4 import BeautifulSoup\n\n# API 설정\nYOUR_API_KEY = \'key\'\ngenai.configure(api_key=YOUR_API_KEY)\n\nst.title("상담Chat: the Counselor for You")\n\n# 모델 로드 함수\n@st.cache_resource\ndef load_model():\n    print("model loaded...")\n    return genai.GenerativeModel(\'gemini-1.5-flash\')\n\nmodel = load_model()\n\n# 대화 세션 초기화\nif "chat_session" not in st.session_state:\n    st.session_state["chat_session"] = model.start_chat(\n        history=[\n            {"ai": "system", "content": "당신은 심리적으로 공감하고 현실적인 대책을 궁리해주는 상담사입니다. 사용자에게 도움을 주세요."}\n        ]\n    )\n\n# 대화 기록 표시\nfor content in st.session_state.chat_session.history:\n    with st.chat_message("ai" if content.role == "model" else "user"):\n        st.markdown(content.parts[0].text)\n\n# 사용자 입력 처리\nif prompt := st.chat_input("무엇이 고민인가요?"):\n    with st.chat_message("user"):\n        st.markdown(prompt)\n    with st.chat_message("ai"):\n        response

In [ ]:
'''
#실행됨
import google.generativeai as genai
import streamlit as st
import requests
from bs4 import BeautifulSoup

# API 설정
YOUR_API_KEY = 'key'
genai.configure(api_key=YOUR_API_KEY)

st.title("상담Chat: the Counselor for You")

# 모델 로드 함수
@st.cache_resource
def load_model():
    generation_config = genai.GenerationConfig(max_output_tokens=500)
    model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
    print("model loaded...")
    return model

model = load_model()

# 대화 세션 초기화 (시스템 메시지 제거)
if "chat_session" not in st.session_state:
    st.session_state["chat_session"] = model.start_chat(history=[{
      "role": "user", "parts": ["당신은 심리적으로 공감하고 현실적인 대책을 궁리해주는 상담사입니다. 사용자에게 도움을 주세요."]
    }])
    st.session_state.messages = []
    st.session_state.language = "한국어"
    st.session_state.stage = 1

# 시스템 메시지 화면에 출력
st.markdown("**시스템:** 안녕하세요! 심리적으로 어려운 점이 있으시면 언제든지 말씀해주세요. 함께 해결해 나갈 수 있도록 노력하겠습니다.")

# 대화 기록 표시
for content in st.session_state.chat_session.history:
    with st.chat_message("ai" if content.role == "model" else "user"):
        st.markdown(content.parts[0].text)

# 사용자 입력 처리
if prompt := st.chat_input("무엇이 고민인가요?"):
    with st.chat_message("user"):
        st.markdown(prompt)
    with st.chat_message("ai"):
        response = st.session_state.chat_session.send_message(prompt)
        st.markdown(response.text)
        '''